In [1]:
import duckdb

In [2]:
# create a connection to a new DuckDB database file
conn = duckdb.connect('my_duckdb_database.db')

In [3]:
conn2 = duckdb.connect(':memory:')

In [6]:
# create a table
conn.execute('''
  CREATE TABLE employees (
      id INTEGER PRIMARY KEY,
      name VARCHAR,
      age INTEGER,
      department VARCHAR
  )'''
)

In [11]:
conn.execute('SHOW TABLES').df()

,name
0,employees


In [19]:
# insert data into the table
conn.execute('''
    INSERT INTO employees VALUES
      (1, 'Alice', 30, 'HR'),
      (2, 'Bob', 35, 'Engineering'),
      (3, 'Charlie', 28, 'Marketing'),
      (4, 'David', 40, 'Engineering')
   '''
)

In [13]:
conn.execute('''
    SELECT * from employees
'''
).df()

,id,name,age,department
0,1,Alice,30,HR
1,2,Bob,35,Enginnering
2,3,Charlie,28,Marketing
3,4,David,40,Engineering


In [20]:
conn.execute('''
  SELECT 
    department,
    COUNT(*) AS employee_count,
  FROM 
    employees
  GROUP BY 
    department
'''    
).df()

,department,employee_count
0,Engineering,2
1,Marketing,1
2,HR,1


In [18]:
conn.execute(
    '''
    TRUNCATE employees
    '''
)

In [21]:
conn.execute('''
  SELECT 
    AVG(age) AS average_age
  FROM employees
'''
).df()

,average_age
0,33.25


In [23]:
conn.execute('''
  SELECT
    department,
    MAX(age) AS oldest_age
  FROM
    employees
  GROUP BY
    department
''').df()

,department,oldest_age
0,Engineering,40
1,Marketing,28
2,HR,30


In [25]:
conn.execute('''
  SELECT
    department,
    AVG(age) AS average_age
  FROM 
    employees
  GROUP BY
    department
''').df()

,department,average_age
0,Engineering,37.5
1,HR,30.0
2,Marketing,28.0
